In [1]:
import pandas as pd
import numpy as np
import os

import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon
import rtree
import pygeos
import fiona
from pykml import parser


pd.options.display.max_columns = 999

/opt/envs/airflow/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
os.getcwd()

'/files/ascollanto/01 BARRM DATA CHALLENGE'

# Administrative Bounds

In [3]:
brgy_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Barangays ao 20190206 (PSA 2016)/AdministrativeBoundariesBARMMBarangays20190206PSA2016.shp")
brgy_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'PSGC_MUNI','MUNI_NAME',
                        'PSGC_BRGY','BRGY_NAME',
                        'geometry']
brgy_bounds["geometry"] = brgy_bounds['geometry'].to_crs(epsg=3857)
brgy_bounds["brgy_area"] = brgy_bounds['geometry'].area/ 10**6

brgy_bounds["geometry"] = brgy_bounds['geometry'].to_crs(epsg=4326)

brgy_bounds['BRGY_NAME'] = brgy_bounds['BRGY_NAME'].str.upper()
brgy_bounds['MUNI_NAME'] = brgy_bounds['MUNI_NAME'].str.upper()
brgy_bounds['PROV_NAME'] = brgy_bounds['PROV_NAME'].str.upper()

# brgy_bounds2 = brgy_bounds[['PSGC_BRGY','geometry']]
# brgy_bounds2.geometry = brgy_bounds2.representative_point()
# brgy_bounds2
brgy_bounds.head()

,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,PSGC_MUNI,MUNI_NAME,PSGC_BRGY,BRGY_NAME,geometry,brgy_area
0,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001001,BALIMBING PROPER,"MULTIPOLYGON (((119.95374 5.07851, 119.95384 5...",15.158874
1,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001002,BATU-BATU (POB.),"POLYGON ((119.88709 5.06919, 119.88709 5.06920...",8.254003
2,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001003,BUAN,"MULTIPOLYGON (((120.06363 5.16131, 120.06365 5...",47.332403
3,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001004,DUNGON,"POLYGON ((120.13585 5.19601, 120.13584 5.19603...",55.353446
4,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH157000000,TAWI-TAWI,PH157001000,PANGLIMA SUGALA (BALIMBING),PH157001005,LUUK BUNTAL,"MULTIPOLYGON (((119.98776 4.98564, 119.98774 4...",6.286992


In [4]:
muni_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Municipalities ao 20190206 (PSA 2016)/AdministrativeBoundariesBARMMMunicipalities20190206PSA2016.shp")
muni_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'PSGC_MUNI','MUNI_NAME',
                        'geometry']

muni_bounds['MUNI_NAME'] = muni_bounds['MUNI_NAME'].str.upper()
muni_bounds['PROV_NAME'] = muni_bounds['PROV_NAME'].str.upper()

muni_bounds["geometry"] = muni_bounds['geometry'].to_crs(epsg=3857)
muni_bounds["muni_area"] = muni_bounds['geometry'].area/ 10**6

muni_bounds["geometry"] = muni_bounds['geometry'].to_crs(epsg=4326)

muni_bounds.head()

,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,PSGC_MUNI,MUNI_NAME,geometry,muni_area
0,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153600000,LANAO DEL SUR,PH153630000,WAO,"POLYGON ((124.82051 7.53628, 124.82037 7.53616...",322.353733
1,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH156600000,SULU,PH156619000,OMAR,"MULTIPOLYGON (((121.38063 5.92761, 121.38063 5...",87.579051
2,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH156600000,SULU,PH156609000,PARANG,"MULTIPOLYGON (((120.88153 6.01489, 120.88092 6...",70.019520
3,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153800000,MAGUINDANAO,PH153807000,DATU ODIN SINSUAT (DINAIG),"POLYGON ((124.29669 6.98930, 124.26466 6.98042...",396.588149
4,PH150000000,AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM),PH153600000,LANAO DEL SUR,PH153621000,PIAGAPO,"POLYGON ((124.10884 7.95196, 124.10220 7.95666...",122.861796


In [5]:
prov_bounds = gpd.read_file("1-Geodata/1-ADMINISTRATIVE/Administrative Boundaries - BARMM Provinces ao 20190206 (PSA2016)/AdministrativeBoundariesBARMMProvinces20190206PSA2016.shp")
prov_bounds.columns = ['PSGC_REGION','REGION_NAME',
                        'PSGC_PROV','PROV_NAME',
                        'geometry']

prov_bounds['PROV_NAME'] = prov_bounds['PROV_NAME'].str.upper()

prov_bounds["geometry"] = prov_bounds['geometry'].to_crs(epsg=3857)
prov_bounds["prov_area"] = prov_bounds['geometry'].area/ 10**6

prov_bounds["geometry"] = prov_bounds['geometry'].to_crs(epsg=4326)

prov_bounds.head()

,PSGC_REGION,REGION_NAME,PSGC_PROV,PROV_NAME,geometry,prov_area
0,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH156600000,SULU,"MULTIPOLYGON (((121.53161 6.01777, 121.53161 6...",1542.357688
1,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH153800000,MAGUINDANAO,"MULTIPOLYGON (((124.04225 7.32152, 124.04225 7...",5030.584700
2,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH124700000,COTABATO (NORTH COTABATO),"MULTIPOLYGON (((124.68982 6.95886, 124.68943 6...",554.957396
3,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH153600000,LANAO DEL SUR,"MULTIPOLYGON (((124.21874 7.89719, 124.21878 7...",3836.238356
4,None,BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO,PH157000000,TAWI-TAWI,"MULTIPOLYGON (((119.46876 4.59360, 119.46881 4...",1171.211004


# Education

In [6]:
schools_2017 = pd.read_excel("2-Tabular Data/2-Infrastructure/SchoolsDepEd2017.xlsx")
schools_2017['SchoolAddress'] = schools_2017['SCH_NAME'] + " " + schools_2017['BARANGAY'] +" "+ schools_2017['MUNNAME'] + " " + schools_2017['PROV']
schools_2017['SchoolAddress'] = schools_2017['SchoolAddress'].replace(" PS", " Primary School", regex=True)
schools_2017['SchoolAddress'] = schools_2017['SchoolAddress'].replace(" ES", " Elementary School", regex=True)

schools_inco = schools_2017.loc[(schools_2017.LAT.isna()) & (schools_2017.LONG.isna())]
schools_comp = schools_2017.loc[(schools_2017.LAT.notna()) & (schools_2017.LONG.notna())]

schools_inco.head()

,SCH_ID,SCH_NAME,SCH_CAT,SCH_TYPE,LAT,LONG,BARANGAY,MUNNAME,PROV,PSGC_MUNI,PSGC_BRGY,PSGC_PROV,SchoolAddress
68,133166,Kinukutan ES,ES,Public,NaN,NaN,Kinukutan,Al-Barka,BASILAN,PH150709000,PH150709009,PH150700000,Kinukutan Elementary School Kinukutan Al-Barka...
157,133059,Upper Mahayahay PS,ES,Public,NaN,NaN,Mahayahay Upper (Zone II),Maluso,BASILAN,PH150704000,PH150704019,PH150700000,Upper Mahayahay Primary School Mahayahay Upper...
203,408468,"Mama's Love Elementary School, Inc.",PES,Private,NaN,NaN,Malakas,CITY OF LAMITAN (Capital),BASILAN,PH150702000,PH150702065,PH150700000,"Mama's Love Elementary School, Inc. Malakas CI..."
206,409135,Madrasatuz Zehra,MD,Private,NaN,NaN,Matatag,CITY OF LAMITAN (Capital),BASILAN,PH150702000,PH150702068,PH150700000,Madrasatuz Zehra Matatag CITY OF LAMITAN (Capi...
324,254505,Datu Caopacatan PS,PS,Public,NaN,NaN,Palacat,Piagapo,LANAO DEL SUR,PH153621000,PH153621022,PH153600000,Datu Caopacatan Primary School Palacat Piagapo...


In [7]:
# Geocode schools that do not have LON LAT
# Run this only once then just read the saved file

# schools_add = pd.DataFrame()
# for i in schools_inco.SchoolAddress:
#     print(str(i)+' out of '+str(schools_inco.shape[0]))
#     lat,long = get_lat_lng(anjAPIKey,i)
#     schools_add = pd.concat([schools_add,pd.DataFrame({'SchoolAddress':[i],'LAT':[lat],'LONG':[long]})])
# schools_add.to_excel('External Source/BARMM_Schools_Incomplete_2017_LonLat.xlsx')

schools_add = pd.read_excel('External Source/BARMM_Schools_Incomplete_2017_LonLat.xlsx').drop(columns = 'Unnamed: 0')
schools_add.head()

,SchoolAddress,LAT,LONG
0,Kinukutan Elementary School Kinukutan Al-Barka...,6.535887,122.161304
1,Upper Mahayahay Primary School Mahayahay Upper...,6.535731,121.885162
2,"Mama's Love Elementary School, Inc. Malakas CI...",6.661519,122.106467
3,Madrasatuz Zehra Matatag CITY OF LAMITAN (Capi...,6.658410,122.133443
4,Datu Caopacatan Primary School Palacat Piagapo...,8.000000,124.199997


In [8]:
schools_inco = schools_inco.drop(columns=['LAT',"LONG"]).merge(schools_add, on = "SchoolAddress", how = 'left')
schools_inco = schools_inco[schools_comp.columns.values]
schools_inco['address_type']  = 'geotagged'

schools_comp['address_type'] = 'given'

schools_2017 = pd.concat([schools_comp,schools_inco])
schools_2017 = gpd.GeoDataFrame(schools_2017,geometry=gpd.points_from_xy(schools_2017.LONG, schools_2017.LAT),crs="EPSG:4326")

schools_2017.head()

/opt/envs/airflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,SCH_ID,SCH_NAME,SCH_CAT,SCH_TYPE,LAT,LONG,BARANGAY,MUNNAME,PROV,PSGC_MUNI,PSGC_BRGY,PSGC_PROV,SchoolAddress,address_type,geometry
0,133049,Abong-Abong ES,ES,Public,6.507703,121.910937,Abong-Abong,Maluso,BASILAN,PH150704000,PH150704001,PH150700000,Abong-Abong Elementary School Abong-Abong Malu...,given,POINT (121.91094 6.50770)
1,215015,Akbar ES,ES,Public,6.664125,122.195725,Upper Bato-Bato,Akbar,BASILAN,PH150708000,PH150708008,PH150700000,Akbar Elementary School Upper Bato-Bato Akbar ...,given,POINT (122.19572 6.66413)
2,133154,Amaloy ES,ES,Public,6.443614,122.126717,Amaloy,Ungkaya Pukan,BASILAN,PH150711000,PH150711001,PH150700000,Amaloy Elementary School Amaloy Ungkaya Pukan ...,given,POINT (122.12672 6.44361)
3,133023,Atong-Atong ES,ES,Public,6.660000,121.899633,Atong-Atong,Lantawan,BASILAN,PH150703000,PH150703001,PH150700000,Atong-Atong Elementary School Atong-Atong Lant...,given,POINT (121.89963 6.66000)
4,133081,Babag ES,ES,Public,6.342007,121.954045,Babag (Babuan Island),Tabuan-Lasa,BASILAN,PH150713000,PH150713001,PH150700000,Babag Elementary School Babag (Babuan Island) ...,given,POINT (121.95404 6.34201)


In [9]:
del schools_inco, schools_comp, schools_add

In [10]:
# check if school lon lat are in barangay
 
schools_2017 = sjoin(schools_2017,brgy_bounds[['geometry','PSGC_BRGY']].rename(columns = {'PSGC_BRGY':'PSGC_BRGY2'}),how='left',op='within').drop(columns=['index_right'])
schools_2017['BRGY_same'] = np.where(schools_2017['PSGC_BRGY'] == schools_2017['PSGC_BRGY2'],1,0)
schools_2017['BRGY_same'].value_counts()
schools_2017.to_csv('Processed/BARMM_Schools.csv')

# students/ schools
# schools per barangay


/opt/envs/airflow/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:32651

  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
base_schools = schools_2017[['SCH_ID','SCH_NAME','SCH_CAT','PSGC_PROV','PSGC_MUNI','PSGC_BRGY','LAT','LONG','address_type','geometry']].drop_duplicates()
base_schools = base_schools.drop(columns = ['SCH_CAT']).drop_duplicates().merge(pd.get_dummies(base_schools[['SCH_ID','SCH_CAT']], prefix=['SCH_CAT'], columns=['SCH_CAT']).groupby('SCH_ID').sum().reset_index(), on = 'SCH_ID', how = 'left')

print(len(set(base_schools.SCH_ID)))
base_schools.head()

2643


,SCH_ID,SCH_NAME,PSGC_PROV,PSGC_MUNI,PSGC_BRGY,LAT,LONG,address_type,geometry,SCH_CAT_CES,SCH_CAT_ES,SCH_CAT_JHS,SCH_CAT_MD,SCH_CAT_PES,SCH_CAT_PJHS,SCH_CAT_PS,SCH_CAT_PSHS,SCH_CAT_SHS,SCH_CAT_SUC
0,133049,Abong-Abong ES,PH150700000,PH150704000,PH150704001,6.507703,121.910937,given,POINT (121.91094 6.50770),0,1,0,0,0,0,0,0,0,0
1,215015,Akbar ES,PH150700000,PH150708000,PH150708008,6.664125,122.195725,given,POINT (122.19572 6.66413),0,1,0,0,0,0,0,0,0,0
2,133154,Amaloy ES,PH150700000,PH150711000,PH150711001,6.443614,122.126717,given,POINT (122.12672 6.44361),0,1,0,0,0,0,0,0,0,0
3,133023,Atong-Atong ES,PH150700000,PH150703000,PH150703001,6.660000,121.899633,given,POINT (121.89963 6.66000),0,1,0,0,0,0,0,0,0,0
4,133081,Babag ES,PH150700000,PH150713000,PH150713001,6.342007,121.954045,given,POINT (121.95404 6.34201),0,1,0,0,0,0,0,0,0,0


## School Types per Brgy, Municipality, Province

In [13]:
school_type_prov = schools_2017.groupby(['PSGC_PROV','SCH_CAT']).agg({"SCH_ID": lambda x: x.nunique()}).reset_index().pivot_table(index=['PSGC_PROV'], columns=['SCH_CAT'], values='SCH_ID').fillna(0).reset_index()
school_type_prov.columns = ['{}{}'.format(c, '' if c in {'PSGC_PROV'} else '_SCH_PROV') for c in school_type_prov.columns]

school_type_muni = schools_2017.groupby(['PSGC_MUNI','SCH_CAT']).agg({"SCH_ID": lambda x: x.nunique()}).reset_index().pivot_table(index=['PSGC_MUNI'], columns=['SCH_CAT'], values='SCH_ID').fillna(0).reset_index()
school_type_muni.columns = ['{}{}'.format(c, '' if c in {'PSGC_MUNI'} else '_SCH_MUNI') for c in school_type_muni.columns]

school_type_brgy = schools_2017.groupby(['PSGC_BRGY','SCH_CAT']).agg({"SCH_ID": lambda x: x.nunique()}).reset_index().pivot_table(index=['PSGC_BRGY'], columns=['SCH_CAT'], values='SCH_ID').fillna(0).reset_index()
school_type_brgy.columns = ['{}{}'.format(c, '' if c in {'PSGC_BRGY'} else '_SCH_BRGY') for c in school_type_brgy.columns]

In [14]:
base_schools = base_schools.merge(school_type_prov,on = "PSGC_PROV",how = 'left')
base_schools = base_schools.merge(school_type_muni,on = "PSGC_MUNI",how = 'left')
base_schools = base_schools.merge(school_type_brgy,on = "PSGC_BRGY",how = 'left')
base_schools.head()

,SCH_ID,SCH_NAME,PSGC_PROV,PSGC_MUNI,PSGC_BRGY,LAT,LONG,address_type,geometry,SCH_CAT_CES,SCH_CAT_ES,SCH_CAT_JHS,SCH_CAT_MD,SCH_CAT_PES,SCH_CAT_PJHS,SCH_CAT_PS,SCH_CAT_PSHS,SCH_CAT_SHS,SCH_CAT_SUC,CES_SCH_PROV,ES_SCH_PROV,JHS_SCH_PROV,MD_SCH_PROV,PES_SCH_PROV,PJHS_SCH_PROV,PS_SCH_PROV,PSHS_SCH_PROV,SHS_SCH_PROV,SUC_SCH_PROV,CES_SCH_MUNI,ES_SCH_MUNI,JHS_SCH_MUNI,MD_SCH_MUNI,PES_SCH_MUNI,PJHS_SCH_MUNI,PS_SCH_MUNI,PSHS_SCH_MUNI,SHS_SCH_MUNI,SUC_SCH_MUNI,CES_SCH_BRGY,ES_SCH_BRGY,JHS_SCH_BRGY,MD_SCH_BRGY,PES_SCH_BRGY,PJHS_SCH_BRGY,PS_SCH_BRGY,PSHS_SCH_BRGY,SHS_SCH_BRGY,SUC_SCH_BRGY
0,133049,Abong-Abong ES,PH150700000,PH150704000,PH150704001,6.507703,121.910937,given,POINT (121.91094 6.50770),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,215015,Akbar ES,PH150700000,PH150708000,PH150708008,6.664125,122.195725,given,POINT (122.19572 6.66413),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,133154,Amaloy ES,PH150700000,PH150711000,PH150711001,6.443614,122.126717,given,POINT (122.12672 6.44361),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,133023,Atong-Atong ES,PH150700000,PH150703000,PH150703001,6.660000,121.899633,given,POINT (121.89963 6.66000),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,18.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,133081,Babag ES,PH150700000,PH150713000,PH150713001,6.342007,121.954045,given,POINT (121.95404 6.34201),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## School Count per Brgy, Municipality, Province

In [15]:
# day_care = gpd.read_file("1-Geodata/5-SOCIAL/Social - Day Care Centers in ARMM (RPDO, 2018)/Social - Day Care Centers, 2018 - ARMM.shp")


In [16]:
schools_per_brgy = pd.read_excel("2-Tabular Data/5-Social/Number of schools per brgy(1).xlsx").rename(columns = {'Number of School':'BRGY_SCHOOLS'})
schools_per_brgy = schools_per_brgy.merge(brgy_bounds[['PSGC_BRGY','brgy_area']], on = 'PSGC_BRGY', how = 'left')
schools_per_brgy['schools_over_brgy_area'] =  schools_per_brgy['BRGY_SCHOOLS']/schools_per_brgy['brgy_area']
schools_per_brgy.head(3)

,PROV,PSGC_PROV,CITY/MUNI,PSGC_CITY/MUNI,BRGY,PSGC_BRGY,BRGY_SCHOOLS,brgy_area,schools_over_brgy_area
0,BASILAN,PH150700000,CITY OF LAMITAN (Capital),PH150702000,Arco,PH150702001,1,9.332738,0.107150
1,BASILAN,PH150700000,CITY OF LAMITAN (Capital),PH150702000,Ba-as,PH150702002,1,2.360844,0.423577
2,BASILAN,PH150700000,CITY OF LAMITAN (Capital),PH150702000,Balagtasan,PH150702005,1,10.019061,0.099810


In [17]:
schools_per_muni = pd.read_excel("2-Tabular Data/5-Social/Number of schools per muni.xlsx").rename(columns={'PSGC_CITY/MUNI':'PSGC_MUNI','Number of School':'MUNI_SCHOOLS'})
schools_per_muni = schools_per_muni.merge(muni_bounds[['PSGC_MUNI','muni_area']], on = 'PSGC_MUNI', how = 'left')
schools_per_muni['schools_over_muni_area'] =  schools_per_muni['MUNI_SCHOOLS']/schools_per_muni['muni_area']
schools_per_muni.head(3)

,PROV,PSGC_PROV,CITY/MUNI,PSGC_MUNI,MUNI_SCHOOLS,muni_area,schools_over_muni_area
0,BASILAN,PH150700000,CITY OF LAMITAN (Capital),PH150702000,61,253.648860,0.240490
1,BASILAN,PH150700000,LANTAWAN,PH150703000,24,152.681422,0.157190
2,BASILAN,PH150700000,MALUSO,PH150704000,20,88.732750,0.225396


In [18]:
schools_per_prov = pd.read_excel("2-Tabular Data/5-Social/Number of School Per Prov.xlsx").rename(columns = {'Number of School':'PROV_SCHOOLS'})
schools_per_prov = schools_per_prov.merge(prov_bounds[['PSGC_PROV','prov_area']], on = 'PSGC_PROV', how = 'left')
schools_per_prov['schools_over_prov_area'] =  schools_per_prov['PROV_SCHOOLS']/schools_per_prov['prov_area']
schools_per_prov.head(3)

,PROV,PSGC_PROV,PROV_SCHOOLS,prov_area,schools_over_prov_area
0,BASILAN,PH150700000,241,1156.417926,0.208402
1,LANAO DEL SUR,PH153600000,978,3836.238356,0.254937
2,MAGUINDANAO,PH153800000,709,5030.584700,0.140938


In [19]:
base_schools = base_schools.merge(schools_per_brgy[['PSGC_BRGY','BRGY_SCHOOLS','brgy_area','schools_over_brgy_area']],on = 'PSGC_BRGY',how='left')
base_schools = base_schools.merge(schools_per_muni[['PSGC_MUNI','MUNI_SCHOOLS','muni_area','schools_over_muni_area']],on = 'PSGC_MUNI',how='left')
base_schools = base_schools.merge(schools_per_prov[['PSGC_PROV','PROV_SCHOOLS','prov_area','schools_over_prov_area']],on = 'PSGC_PROV',how='left')
base_schools.head()

,SCH_ID,SCH_NAME,PSGC_PROV,PSGC_MUNI,PSGC_BRGY,LAT,LONG,address_type,geometry,SCH_CAT_CES,SCH_CAT_ES,SCH_CAT_JHS,SCH_CAT_MD,SCH_CAT_PES,SCH_CAT_PJHS,SCH_CAT_PS,SCH_CAT_PSHS,SCH_CAT_SHS,SCH_CAT_SUC,CES_SCH_PROV,ES_SCH_PROV,JHS_SCH_PROV,MD_SCH_PROV,PES_SCH_PROV,PJHS_SCH_PROV,PS_SCH_PROV,PSHS_SCH_PROV,SHS_SCH_PROV,SUC_SCH_PROV,CES_SCH_MUNI,ES_SCH_MUNI,JHS_SCH_MUNI,MD_SCH_MUNI,PES_SCH_MUNI,PJHS_SCH_MUNI,PS_SCH_MUNI,PSHS_SCH_MUNI,SHS_SCH_MUNI,SUC_SCH_MUNI,CES_SCH_BRGY,ES_SCH_BRGY,JHS_SCH_BRGY,MD_SCH_BRGY,PES_SCH_BRGY,PJHS_SCH_BRGY,PS_SCH_BRGY,PSHS_SCH_BRGY,SHS_SCH_BRGY,SUC_SCH_BRGY,BRGY_SCHOOLS,brgy_area,schools_over_brgy_area,MUNI_SCHOOLS,muni_area,schools_over_muni_area,PROV_SCHOOLS,prov_area,schools_over_prov_area
0,133049,Abong-Abong ES,PH150700000,PH150704000,PH150704001,6.507703,121.910937,given,POINT (121.91094 6.50770),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,4.590691,0.435664,20,88.732750,0.225396,241,1156.417926,0.208402
1,215015,Akbar ES,PH150700000,PH150708000,PH150708008,6.664125,122.195725,given,POINT (122.19572 6.66413),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.797243,1.112815,7,31.433729,0.222691,241,1156.417926,0.208402
2,133154,Amaloy ES,PH150700000,PH150711000,PH150711001,6.443614,122.126717,given,POINT (122.12672 6.44361),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.478142,0.287510,13,71.247126,0.182463,241,1156.417926,0.208402
3,133023,Atong-Atong ES,PH150700000,PH150703000,PH150703001,6.660000,121.899633,given,POINT (121.89963 6.66000),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,18.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5.649323,0.354025,24,152.681422,0.157190,241,1156.417926,0.208402
4,133081,Babag ES,PH150700000,PH150713000,PH150713001,6.342007,121.954045,given,POINT (121.95404 6.34201),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.570479,0.389033,16,28.023131,0.570957,241,1156.417926,0.208402


# Population Variables

In [34]:
# Population_2010_2015 = pd.read_excel("2-Tabular Data/5-Social/Population 2010, 2015 ARMM,BCT.xlsx")
# Population_2010_2015 = Population_2010_2015.loc[Population_2010_2015.PSGC_REGI=="PH150000000"]


Population_2010_2015['Total_under10'] = Population_2010_2015.loc[:, 'sin_age_bot_und_001':'sin_age_bot_009'].sum(axis=1)
Population_2010_2015['Total_10to20'] = Population_2010_2015.loc[:, 'sin_age_bot_010':'sin_age_bot_020'].sum(axis=1)
Population_2010_2015['Total_21to30'] = Population_2010_2015.loc[:, 'sin_age_bot_021':'sin_age_bot_030'].sum(axis=1)
Population_2010_2015['Total_31to60'] = Population_2010_2015.loc[:, 'sin_age_bot_031':'sin_age_bot_059'].sum(axis=1)
Population_2010_2015['Total_over60'] = Population_2010_2015.loc[:, 'sin_age_bot_060':'sin_age_bot_080_and_ove'].sum(axis=1)


Population_2010_2015['Male_under10'] = Population_2010_2015.loc[:, 'sin_age_mal_und_001':'sin_age_mal_009'].sum(axis=1)
Population_2010_2015['Male_10to20'] = Population_2010_2015.loc[:, 'sin_age_mal_010':'sin_age_mal_020'].sum(axis=1)
Population_2010_2015['Male_21to30'] = Population_2010_2015.loc[:, 'sin_age_mal_021':'sin_age_mal_030'].sum(axis=1)
Population_2010_2015['Male_31to60'] = Population_2010_2015.loc[:, 'sin_age_mal_031':'sin_age_mal_059'].sum(axis=1)
Population_2010_2015['Male_over60'] = Population_2010_2015.loc[:, 'sin_age_mal_060':'sin_age_mal_080_and_ove'].sum(axis=1)

Population_2010_2015['Female_under10'] = Population_2010_2015.loc[:, 'sin_age_fem_und_001':'sin_age_fem_009'].sum(axis=1)
Population_2010_2015['Female_10to20'] = Population_2010_2015.loc[:, 'sin_age_fem_010':'sin_age_fem_020'].sum(axis=1)
Population_2010_2015['Female_21to30'] = Population_2010_2015.loc[:, 'sin_age_fem_021':'sin_age_fem_030'].sum(axis=1)
Population_2010_2015['Female_31to60'] = Population_2010_2015.loc[:, 'sin_age_fem_031':'sin_age_fem_059'].sum(axis=1)
Population_2010_2015['Female_over60'] = Population_2010_2015.loc[:, 'sin_age_fem_060':'sin_age_fem_080_and_ove'].sum(axis=1)

Population_2010_2015['Total_Pop'] = Population_2010_2015['Tot_sin_age_bot']
Population_2010_2015['Male_Pop'] = Population_2010_2015['tot_sin_age_mal']
Population_2010_2015['Female_Pop'] = Population_2010_2015['tot_sin_age_fem']

Population_2010_2015 = Population_2010_2015[['PSGC_CITY/MUNI','PSGC_BRGY',
                                             'Pop2010','Pop2015',
                                             'HH_pop', 'num_hh0', 'num_hh0_mal', 'num_hh0_fem',
                                             'Total_Pop','Male_Pop','Female_Pop',
                                             'Total_under10','Total_10to20','Total_21to30','Total_31to60','Total_over60',
                                             'Female_under10','Female_10to20','Female_21to30','Female_31to60','Female_over60',
                                             'Male_under10','Male_10to20','Male_21to30','Male_31to60','Male_over60']].rename(columns = {'PSGC_CITY/MUNI':'PSGC_MUNI'})
Population_2010_2015
# ethnicity = pd.read_excel("2-Tabular Data/5-Social/Household Population by Ethnicity (ARMM, 2010)/Household Population by Ethnicity (ARMM, 2010) Brgy.xlsx") #household level i think
# ethnicity.loc[:, (ethnicity != 0).any(axis=0)]
# # Areas that have restrictions in entering
# ground_zero = pd.read_excel("2-Tabular Data/1-Administrative/Ground Zero.xlsx")

,PSGC_MUNI,PSGC_BRGY,Pop2010,Pop2015,HH_pop,num_hh0,num_hh0_mal,num_hh0_fem,Total_Pop,Male_Pop,Female_Pop,Total_under10,Total_10to20,Total_21to30,Total_31to60,Total_over60,Female_under10,Female_10to20,Female_21to30,Female_31to60,Female_over60,Male_under10,Male_10to20,Male_21to30,Male_31to60,Male_over60
0,PH150702000,PH150702001,800,672,667.0,160.0,137.0,23.0,672.0,360.0,312.0,167.0,140.0,122.0,194.0,49.0,78.0,62.0,54.0,92.0,26.0,89.0,78.0,68.0,102.0,23.0
1,PH150702000,PH150702002,1266,1255,1255.0,231.0,159.0,72.0,1255.0,596.0,659.0,371.0,308.0,203.0,292.0,81.0,194.0,157.0,99.0,158.0,51.0,177.0,151.0,104.0,134.0,30.0
2,PH150702000,PH150702004,451,500,500.0,101.0,76.0,25.0,500.0,241.0,259.0,119.0,158.0,70.0,120.0,33.0,52.0,84.0,42.0,62.0,19.0,67.0,74.0,28.0,58.0,14.0
3,PH150702000,PH150702005,2597,2359,2359.0,434.0,358.0,76.0,2359.0,1196.0,1163.0,582.0,566.0,409.0,679.0,123.0,271.0,274.0,211.0,350.0,57.0,311.0,292.0,198.0,329.0,66.0
4,PH150702000,PH150702006,2809,2918,2918.0,546.0,380.0,166.0,2918.0,1485.0,1433.0,706.0,771.0,532.0,745.0,164.0,359.0,357.0,256.0,381.0,80.0,347.0,414.0,276.0,364.0,84.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,PH157011000,PH157011012,2326,2509,2509.0,377.0,351.0,26.0,2509.0,1239.0,1270.0,578.0,693.0,395.0,731.0,112.0,299.0,349.0,201.0,373.0,48.0,279.0,344.0,194.0,358.0,64.0
2478,PH157011000,PH157011013,2610,3050,3050.0,472.0,445.0,27.0,3050.0,1549.0,1501.0,700.0,705.0,631.0,885.0,129.0,347.0,353.0,294.0,443.0,64.0,353.0,352.0,337.0,442.0,65.0
2479,PH157011000,PH157011014,1433,1559,1559.0,303.0,288.0,15.0,1559.0,805.0,754.0,393.0,367.0,279.0,469.0,51.0,172.0,185.0,149.0,227.0,21.0,221.0,182.0,130.0,242.0,30.0
2480,PH157011000,PH157011015,2016,2216,2216.0,322.0,303.0,19.0,2216.0,1127.0,1089.0,513.0,573.0,419.0,595.0,116.0,262.0,284.0,199.0,294.0,50.0,251.0,289.0,220.0,301.0,66.0


In [37]:
base_schools = base_schools.merge(Population_2010_2015, on = ['PSGC_MUNI','PSGC_BRGY'], how ='left')
base_schools.head()

,SCH_ID,SCH_NAME,PSGC_PROV,PSGC_MUNI,PSGC_BRGY,LAT,LONG,address_type,geometry,SCH_CAT_CES,SCH_CAT_ES,SCH_CAT_JHS,SCH_CAT_MD,SCH_CAT_PES,SCH_CAT_PJHS,SCH_CAT_PS,SCH_CAT_PSHS,SCH_CAT_SHS,SCH_CAT_SUC,CES_SCH_PROV,ES_SCH_PROV,JHS_SCH_PROV,MD_SCH_PROV,PES_SCH_PROV,PJHS_SCH_PROV,PS_SCH_PROV,PSHS_SCH_PROV,SHS_SCH_PROV,SUC_SCH_PROV,CES_SCH_MUNI,ES_SCH_MUNI,JHS_SCH_MUNI,MD_SCH_MUNI,PES_SCH_MUNI,PJHS_SCH_MUNI,PS_SCH_MUNI,PSHS_SCH_MUNI,SHS_SCH_MUNI,SUC_SCH_MUNI,CES_SCH_BRGY,ES_SCH_BRGY,JHS_SCH_BRGY,MD_SCH_BRGY,PES_SCH_BRGY,PJHS_SCH_BRGY,PS_SCH_BRGY,PSHS_SCH_BRGY,SHS_SCH_BRGY,SUC_SCH_BRGY,BRGY_SCHOOLS,brgy_area,schools_over_brgy_area,MUNI_SCHOOLS,muni_area,schools_over_muni_area,PROV_SCHOOLS,prov_area,schools_over_prov_area,Pop2010,Pop2015,HH_pop,num_hh0,num_hh0_mal,num_hh0_fem,Total_Pop,Male_Pop,Female_Pop,Total_under10,Total_10to20,Total_21to30,Total_31to60,Total_over60,Female_under10,Female_10to20,Female_21to30,Female_31to60,Female_over60,Male_under10,Male_10to20,Male_21to30,Male_31to60,Male_over60
0,133049,Abong-Abong ES,PH150700000,PH150704000,PH150704001,6.507703,121.910937,given,POINT (121.91094 6.50770),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,4.590691,0.435664,20,88.732750,0.225396,241,1156.417926,0.208402,846.0,965.0,965.0,176.0,155.0,21.0,965.0,473.0,492.0,297.0,245.0,152.0,222.0,49.0,144.0,134.0,85.0,104.0,25.0,153.0,111.0,67.0,118.0,24.0
1,215015,Akbar ES,PH150700000,PH150708000,PH150708008,6.664125,122.195725,given,POINT (122.19572 6.66413),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1.797243,1.112815,7,31.433729,0.222691,241,1156.417926,0.208402,642.0,1093.0,1093.0,175.0,135.0,40.0,1093.0,520.0,573.0,300.0,303.0,134.0,308.0,48.0,134.0,153.0,72.0,185.0,29.0,166.0,150.0,62.0,123.0,19.0
2,133154,Amaloy ES,PH150700000,PH150711000,PH150711001,6.443614,122.126717,given,POINT (122.12672 6.44361),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.478142,0.287510,13,71.247126,0.182463,241,1156.417926,0.208402,980.0,1129.0,1129.0,191.0,187.0,4.0,1129.0,586.0,543.0,329.0,284.0,154.0,318.0,44.0,136.0,143.0,91.0,153.0,20.0,193.0,141.0,63.0,165.0,24.0
3,133023,Atong-Atong ES,PH150700000,PH150703000,PH150703001,6.660000,121.899633,given,POINT (121.89963 6.66000),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,18.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,5.649323,0.354025,24,152.681422,0.157190,241,1156.417926,0.208402,1155.0,2368.0,2368.0,419.0,367.0,52.0,2368.0,1173.0,1195.0,594.0,621.0,377.0,661.0,115.0,297.0,295.0,196.0,342.0,65.0,297.0,326.0,181.0,319.0,50.0
4,133081,Babag ES,PH150700000,PH150713000,PH150713001,6.342007,121.954045,given,POINT (121.95404 6.34201),0,1,0,0,0,0,0,0,0,0,16.0,171.0,16.0,1.0,6.0,0.0,14.0,6.0,10.0,1.0,1.0,11.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.570479,0.389033,16,28.023131,0.570957,241,1156.417926,0.208402,1799.0,2179.0,2179.0,313.0,301.0,12.0,2179.0,1083.0,1096.0,569.0,768.0,320.0,469.0,53.0,280.0,392.0,155.0,239.0,30.0,289.0,376.0,165.0,230.0,23.0


In [36]:
base_schools.to_csv("Processed/0_Education_ForModelling.csv")